In [13]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

from experiment_utils import *
from src import *


# SAVE TO RESULTS FILE

In [14]:
FOLDER = "RQ4_CONSISTENCY"

if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)

In [15]:
"""
For all baselines and my relavant losses count of times the metric has been equal or higher than the baseline model
# TODO: WHAT IF INSTEAD I CHECK SIGNIFICANTLY LESS AND SIGNIFICANTLY MORE??????
"""

'\nFor all baselines and my relavant losses count of times the metric has been equal or higher than the baseline model\n# TODO: WHAT IF INSTEAD I CHECK SIGNIFICANTLY LESS AND SIGNIFICANTLY MORE??????\n'

In [16]:
for attr in [["sex"],["race"],["race", "sex"]]:
#for attr in [["race", "sex"]]:
    files = [os.path.join("results","RESULTS_"+'_'.join(attr)+".csv")]
    reader = ResultsReader(files)
    reader.change_other_to_losses()

    rel_df = reader.get_relative_metrics()

    metrics_to_use = ["f1", "MCC", "A_", "DI_FM", "SPD", "SF", "DF"]
    metrics_to_display = [m for m in reader.metrics if metric_has_substring(m,metrics_to_use )]
    
    for data in ["Adult", "Compas"]:        
        df = rel_df[(rel_df["data"].str.contains(data))]
        df = df[(~df["bias mitigation"].str.contains('FYP VAE')) | df["other"].str.contains('Sensitive KL loss') & ~df["other"].str.contains('Flipped') ]
        
        if df.shape[0] > 0: # in case some dataset has no results for some metrics
            counts_total = df[(df["bias mitigation"].str.contains('No'))].groupby(["bias mitigation", "other"]).count()[metrics_to_display]
            c = counts_total["MCC"].iloc[0]
            print(data," count ", c)
            result = df.groupby(["bias mitigation", "other"])[metrics_to_display].apply(lambda x: (x >= 0).sum()).reset_index()
            results = pd.concat([result, counts_total])
            
            results.to_csv(os.path.join(FOLDER ,'PER_METRIC_nr_incr_'+data+'_'+"_".join(attr)+'.csv'), index=False)

File 'results\RESULTS_sex.csv' successfully loaded as DataFrame.
Adult  count  240
Compas  count  240
File 'results\RESULTS_race.csv' successfully loaded as DataFrame.
Adult  count  240
Compas  count  240
File 'results\RESULTS_race_sex.csv' successfully loaded as DataFrame.
Adult  count  1
Compas  count  240


In [17]:
"""
For all baselines and my relavant losses count of times SF has been equal or higher than the baseline model, separately for each of the 6 ml models used
"""

'\nFor all baselines and my relavant losses count of times SF has been equal or higher than the baseline model, separately for each of the 6 ml models used\n'

In [18]:
# per diff ml model calc the nr of times  for a given metric

for attr in [["sex"],["race"],["race", "sex"]]:
    files = [os.path.join("results","RESULTS_"+'_'.join(attr)+".csv")]
    reader = ResultsReader(files)
    reader.change_other_to_losses()

    rel_df = reader.get_relative_metrics()

    metrics_to_use = ["SF"]
    metrics_to_display = [m for m in reader.metrics if metric_has_substring(m,metrics_to_use )]
            
    for data in ["Adult", "Compas"]:
        df = rel_df[(rel_df["data"].str.contains(data))]
        df = df[(~df["bias mitigation"].str.contains('FYP VAE')) | df["other"].str.contains('Sensitive KL loss') & ~df["other"].str.contains('Flipped') ]
        
        if df.shape[0] > 0: # in case some dataset has no results for some metrics
            counts_total = df[(df["bias mitigation"].str.contains('No'))].groupby(["bias mitigation", "other"]).count()[metrics_to_display]
            result = df.groupby(["bias mitigation", "other", "ML method"])[metrics_to_display].apply(lambda x: (x >= 0).sum()).reset_index()
            results = pd.concat([result, counts_total])
            
            pivot_df = results.pivot_table(index=["bias mitigation", "other"], columns='ML method', values=metrics_to_display[0], aggfunc='sum', fill_value=0)            
            pivot_df.to_csv(os.path.join(FOLDER,'PER_MODEL_nr_incr_'+data+'_'+"_".join(attr)+'.csv'), index=True,  index_label=["bias mitigation", "other"])

File 'results\RESULTS_sex.csv' successfully loaded as DataFrame.
